This notebook goes through the voice conversion of speakers from the ARCTIC corpus using the SIDEKIT toolkit for feature extraction.

## Set up

In [1]:
%set_env SIDEKIT=theano=false
import sidekit
import os
from os import listdir

env: SIDEKIT=theano=false


/Users/kennylino/.local/share/virtualenvs/thesis-W2ZHkp3l/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
!pwd

/Users/kennylino/Documents/em_lct/UoM/thesis/experiments


In [3]:
# if cwd not './thesis' (e.g. './thesis/experiments/'), moves one up
os.chdir('..')
!pwd

/Users/kennylino/Documents/em_lct/UoM/thesis


In [4]:
# takes in a list of speaker directories
# returns a single list containing links to each audio file from each speaker
def create_ubm_data(corpus_dir, speaker_dir):
    ubm_data = []
    for speaker in speaker_dir:
        file_list = os.listdir(corpus_dir + speaker) # list of file names
        speaker_data = [corpus_dir + speaker + os.path.split(f)[0] for f in file_list if f != '.DS_Store']
        ubm_data.extend(speaker_data)
    return (ubm_data)

In [5]:
# ubm file list
corpus_dir = "./data/arctic/" # corpus directory
speaker_dir = ['cmu_us_awb_arctic/wav/', 'cmu_us_clb_arctic/wav/'] # speaker data directories
ubm_data = create_ubm_data(corpus_dir, speaker_dir)

In [6]:
ubm_data[0]

'./data/arctic/cmu_us_awb_arctic/wav/'

## Extract features

http://www-lium.univ-lemans.fr/sidekit/tutorial/hdf5.html

In [7]:
# defines how to extract features (MFCCs, bottleneck, etc.) of audio files in a given path
extractor = sidekit.FeaturesExtractor(audio_filename_structure='./{}.wav',
                                      feature_filename_structure=None,
                                      sampling_frequency=16000,
                                      lower_frequency=200,
                                      higher_frequency=3800,
                                      filter_bank="log",
                                      filter_bank_size=24,
                                      window_size=0.025,
                                      shift=0.01,
                                      ceps_number=13,
                                      vad="snr",
                                      snr=40,
                                      pre_emphasis=0.97,
                                      save_param=["vad", "energy", "cep"],
                                      keep_all_features=True)

## UBM-GMM Model

http://www-lium.univ-lemans.fr/sidekit/tutorial/ubmTraining.html

In [8]:
# defines a FeaturesServer that describes *how* to load the data
fs = sidekit.FeaturesServer(features_extractor=extractor,
                             feature_filename_structure=None,
                             sources=None,
                             dataset_list=["vad", "energy", "cep"],
                             mask=None,
                             feat_norm="cmvn",
                             global_cmvn=None,
                             dct_pca=False,
                             dct_pca_config=None,
                             sdc=False,
                             sdc_config=None,
                             delta=True,
                             double_delta=True,
                             delta_filter=None,
                             context=None,
                             traps_dct_nb=None,
                             rasta=False,
                             keep_all_features=False)

In [9]:
!pwd

/Users/kennylino/Documents/em_lct/UoM/thesis


In [ ]:
ubm = sidekit.Mixture()
model = ubm.EM_split(features_server=fs,
                      feature_list=ubm_data[0:1],
                      distrib_nb=64,
                      iterations=(1, 2, 2, 4, 4, 4, 4),
                      save_partial=False,
                      ceil_cov=10,
                      floor_cov=1e-2
                      )
ubm.write(corpus_dir + 'ubm.h5')

Process Process-1:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/kennylino/.local/share/virtualenvs/thesis-W2ZHkp3l/lib/python3.6/site-packages/sidekit/features_server.py", line 635, in _stack_features_worker
    output_queue.put(self.load(*next_task)[0])
  File "/Users/kennylino/.local/share/virtualenvs/thesis-W2ZHkp3l/lib/python3.6/site-packages/sidekit/features_server.py", line 438, in load
    start=start, stop=stop)
  File "/Users/kennylino/.local/share/virtualenvs/thesis-W2ZHkp3l/lib/python3.6/site-packages/sidekit/features_server.py", line 476, in get_features
    h5f = self.features_extractor.extract(show, channel, input_audio_filena